In [2]:
from gbmbkgpy.utils.continuous_data import ContinuousData 
from gbmbkgpy.utils.external_prop import ExternalProps
from gbmbkgpy.point_source import PointSource
from gbmbkgpy.modeling.model import Model
from gbmbkgpy.modeling.source import Source, ContinuumSource, FlareSource, PointSource, SAASource
from gbmbkgpy.modeling.function import Function, ContinuumFunction
from gbmbkgpy.functions import (Solar_Flare, Solar_Continuum, SAA_Decay,
Magnetic_Continuum, Cosmic_Gamma_Ray_Background, Point_Source_Continuum, Earth_Albedo_Continuum)



from gbmbkgpy.io.package_data import get_path_of_data_dir, get_path_of_data_file
from gbmbkgpy.utils.progress_bar import progress_bar
from gbmbkgpy.io.plotting.step_plots import step_plot, slice_disjoint, disjoint_patch_plot

import scipy.interpolate as interpolate
import astropy.units as u
from gbmgeometry import GBMTime
import astropy.time as astro_time
import astropy.coordinates as coord
import astropy.units as u
import astropy.io.fits as fits
import numpy as np
import collections
import matplotlib.pyplot as plt
import math

In [3]:
cd = ContinuousData('/home/felix/Documents/projects/bachelor-thesis/data/ctime/150914/glg_ctime_n5_150914_v00.pha','/home/felix/Documents/projects/bachelor-thesis/data/poshist/glg_poshist_all_150914_v00.fit')
ep = ExternalProps('150914')

/home/felix/Documents/projects/bachelor-thesis/GBM-background-model/gbmbkgpy/utils/continuous_data.py:431: RuntimeWarning: invalid value encountered in sqrt
  np.tan(beta)) - np.cos(opening_ang) * np.arccos(np.tan(beta) / np.tan(opening_ang)) - (np.arctan2(


/home/felix/Documents/projects/bachelor-thesis/GBM-background-model/gbmbkgpy/utils/continuous_data.py:431: RuntimeWarning: invalid value encountered in arccos
  np.tan(beta)) - np.cos(opening_ang) * np.arccos(np.tan(beta) / np.tan(opening_ang)) - (np.arctan2(


/home/felix/Documents/projects/bachelor-thesis/GBM-background-model/gbmbkgpy/utils/continuous_data.py:439: RuntimeWarning: invalid value encountered in sqrt
  np.tan(beta_e)) - np.cos(theta) * np.arccos(np.tan(beta_e) / np.tan(theta)) - (


/home/felix/Documents/projects/bachelor-thesis/GBM-background-model/gbmbkgpy/utils/continuous_data.py:439: RuntimeWarning: invalid value encountered in arccos
  np.tan(beta_e)) - np.cos(theta) * np.arccos(np.tan(beta_e) / np.tan(theta)) - (


/home/felix/Documents/projects/bachelor-thesis/GBM-background-model/gbmbkgpy/utils/continuous_data.py:432: RuntimeWarning: invalid value encountered in sqrt
  (np.sqrt(-(np.tan(beta)) ** 2 / ((np.sin(beta)) ** 2) + (np.tan(beta)) ** 2 + 1) * np.sin(beta)),
/home/felix/Documents/projects/bachelor-thesis/GBM-background-model/gbmbkgpy/utils/continuous_data.py:441: RuntimeWarning: invalid value encountered in sqrt
  -(np.tan(beta_e)) ** 2 / ((np.sin(beta_e)) ** 2) + (np.tan(beta_e)) ** 2 + 1) * np.sin(beta_e)),


In [4]:

import sys

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

[('PS_Continuum_dic', 1048),
 ('ContinuousData', 904),
 ('ContinuumFunction', 904),
 ('ContinuumSource', 904),
 ('Cosmic_Gamma_Ray_Background', 904),
 ('Earth_Albedo_Continuum', 904),
 ('ExternalProps', 904),
 ('FlareSource', 904),
 ('Function', 904),
 ('GBMTime', 904),
 ('Magnetic_Continuum', 904),
 ('Model', 904),
 ('NamespaceMagics', 904),
 ('PointSource', 904),
 ('Point_Source_Continuum', 904),
 ('SAASource', 904),
 ('SAA_Decay', 904),
 ('Solar_Continuum', 904),
 ('Solar_Flare', 904),
 ('Source', 904),
 ('PS_Sources_list', 200),
 ('disjoint_patch_plot', 120),
 ('get_path_of_data_dir', 120),
 ('get_path_of_data_file', 120),
 ('progress_bar', 120),
 ('slice_disjoint', 120),
 ('step_plot', 120),
 ('var_dic_list', 120),
 ('getsizeof', 72),
 ('cd', 64),
 ('ep', 64),
 ('ps', 64),
 ('astro_time', 56),
 ('coord', 56),
 ('fits', 56),
 ('interpolate', 56),
 ('np', 56),
 ('plt', 56),
 ('u', 56)]

In [3]:
# Point-Source Sources
ep.build_point_sources(cd)
PS_Continuum_dic = {}
PS_Sources_list = []

for ps in ep.point_sources.itervalues():
    PS_Continuum_dic[ps.name] = Point_Source_Continuum()
    PS_Continuum_dic[ps.name].set_interpolated_function(ps.separation_angle)

    PS_Sources_list.append(PointSource(ps.name, PS_Continuum_dic[ps.name]))

#add *PS_Sources_list to model constructor

In [ ]:
# SAA Decay Source
SAA_Decay_list = []
saa_n = 0

for time in cd.saa_mean_times:
    saa_n += 1
    saa_dec = SAA_Decay()
    saa_dec.set_saa_exit_time(np.array([time]))
    SAA_Decay_list.append(SAASource('saa_' + str(saa_n), saa_dec))

In [3]:
# Solar Continuum Source
sol_con = Solar_Continuum()
sol_con.set_interpolated_function(cd.effective_angle(4))
Source_Solar_Continuum = ContinuumSource('solar_continuum', sol_con)

In [4]:
# Magnetic Continuum Source
mag_con = Magnetic_Continuum()
mag_con.set_interpolated_function(ep.mc_l)
Source_Magnetic_Continuum = ContinuumSource('magnetic_continuum', mag_con)

In [5]:
# Earth Albedo Continuum Source
earth_albedo = Earth_Albedo_Continuum()
earth_albedo.set_interpolated_function(cd.effective_area(4))
Source_Earth_Albedo_Continuum = ContinuumSource('earth_albedo_continuum', earth_albedo)

In [6]:
# Cosmic gamma-ray background
cgb = Cosmic_Gamma_Ray_Background()
cgb.set_interpolated_function(cd.cgb_background)
Source_CGB_Continuum = ContinuumSource('cgb_continuum', cgb)

In [7]:
#Source_list = PS_Sources_list + SAA_Decay_list
model = Model(Source_Solar_Continuum, Source_Magnetic_Continuum)#, *Source_list)

NameError: name 'Source_Solar_Continuum' is not defined

In [ ]:
model.get_continuum_flux(1, 463881899.39806604,463882490.76702607)

In [ ]:
model.continuum_sources.values()[1].get_flux(463881899.39806604,463882490.76702607)

In [ ]:
for source in model._continuum_sources.values():
    print source.get_flux(463881899.39806604,463882490.76702607)

In [ ]:
model._continuum_sources.values()


In [ ]:
false_count = 0
true_count = 0
for i in cd.saa_mask:
    if i == False:
        false_count +=1
    else: true_count += 1
print true_count
print false_count

In [ ]:
len(cd.time_bins[cd.saa_mask])

In [ ]:
len(cd.time_bins)

In [ ]:
len(cd.counts)

In [ ]:
len(cd.interpolation_time)

In [ ]:
PS_Sources_list[1]._occulted_time

In [ ]:
b = np.array(ep.point_sources['1E1740-29']._src_occ_ang)

In [5]:
a = np.array(ep.point_sources['1E1740-29']._occulted_time)


In [6]:
a

array([  4.63881604e+08,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,

In [ ]:
for i in range (100, 200):
    b[i] = 0.

In [ ]:
a[np.where(b != 0)] = 0

In [ ]:
np.where(b == 0.)

In [ ]:
a

In [4]:
cd.n_channels

8

# SUM of fluxes
sum_solar_con = 0.
sum_magnetic_con = 0.

    
with progress_bar(len(cd.time_bins[2:-2]),title='Calculating solar flux') as p:

    for bin in cd.time_bins[2:-2]:  
        sum_solar_con += Source_Solar_Continuum.get_flux(bin[0], bin[1])
        p.increase()
with progress_bar(len(cd.time_bins[2:-2]),title='Calculating magnetic flux') as p:

    for bin in cd.time_bins[2:-2]:  
        sum_magnetic_con += Source_Magnetic_Continuum.get_flux(bin[0], bin[1])
        p.increase()
        
print "Solar sum: {}".format(sum_solar_con)
print "Magnetic sum {}".format(sum_magnetic_con)

In [ ]:
arr_solar_con = []
arr_solar_con_quad = []
arr_magnetic_con = []
arr_magnetic_con_quad = []

    
with progress_bar(len(cd.time_bins[2:-2]),title='Calculating solar flux') as p:
    for i, bin in enumerate(cd.time_bins[2:-2]): 
        if Source_Solar_Continuum.get_flux(bin[0], bin[1]) < 38.:
            #mean_val_sol = 0.5*(Source_Solar_Continuum.get_flux(cd.time_bins[i-1][0], cd.time_bins[i-1][1]) + \
                           #Source_Solar_Continuum.get_flux(cd.time_bins[i+1][0], cd.time_bins[i-+1][1]))
            #arr_solar_con.append(mean_val_sol)
            arr_solar_con.append(Source_Solar_Continuum.get_flux(bin[0], bin[1]))
            #print "Bin: {0} - {1} results in Int Value: {2}".format(bin[0], bin[1], Source_Solar_Continuum.get_flux(bin[0], bin[1]) )
        else:
            arr_solar_con.append(Source_Solar_Continuum.get_flux(bin[0], bin[1]))
        #arr_solar_con_quad.append(Source_Solar_Continuum.get_flux_quad(bin[0], bin[1]))
        
        p.increase()
with progress_bar(len(cd.time_bins[2:-2]),title='Calculating magnetic flux') as p:

    for i, bin in enumerate(cd.time_bins[2:-2]):  
        #arr_magnetic_con.append(Source_Magnetic_Continuum.get_flux(bin[0], bin[1]))
        #arr_magnetic_con_quad.append(Source_Magnetic_Continuum.get_flux_quad(bin[0], bin[1]))
        if Source_Magnetic_Continuum.get_flux(bin[0], bin[1]) < 0.2:
            #mean_val_mag = 0.5*(Source_Magnetic_Continuum.get_flux(cd.time_bins[i-1][0], cd.time_bins[i-1][1]) + \
                            #Source_Magnetic_Continuum.get_flux(cd.time_bins[i+1][0], cd.time_bins[i+1][1]))
            #print "Bin: {0} - {1} results in Int Value: {2}".format(bin[0], bin[1], Source_Magnetic_Continuum.get_flux(bin[0], bin[1]))
            #arr_magnetic_con.append(mean_val_mag)
            arr_magnetic_con.append(Source_Magnetic_Continuum.get_flux(bin[0], bin[1]))
        else:
            arr_magnetic_con.append(Source_Magnetic_Continuum.get_flux(bin[0], bin[1]))
        p.increase()

solar_con_interpolator = interpolate.interp1d(cd.mean_time[2:-2], arr_solar_con)
magnetic_con_interpolator = interpolate.interp1d(cd.mean_time[2:-2], arr_magnetic_con)

In [ ]:
plt.figure(figsize=(20,20))

plt.subplot(321)
plt.plot(cd.mean_time[2:-2],  cd.sun_angle(cd.mean_time[2:-2]))
plt.title('sun angle')

plt.subplot(322)
plt.plot(cd.mean_time[2:-2],  ep.mc_l(cd.mean_time[2:-2]))
plt.title('mc-l')

plt.subplot(323)  #sublot(Anzahl Zeilen Anzahl Spalten Bild Nummer)
plt.plot(cd.mean_time[2:-2], solar_con_interpolator(cd.mean_time[2:-2]))
plt.title('solar_flux')

plt.subplot(324)
plt.plot(cd.mean_time[2:-2], magnetic_con_interpolator(cd.mean_time[2:-2]))
plt.title('magnetic_flux')

plt.subplot(325)  #sublot(Anzahl Zeilen Anzahl Spalten Bild Nummer)
plt.plot(cd.mean_time[2:-2], np.zeros_like(cd.mean_time[2:-2])) #arr_solar_con_quad)
plt.title('solar_flux_quad')

plt.subplot(326)
plt.plot(cd.mean_time[2:-2], np.zeros_like(cd.mean_time[2:-2])) # arr_magnetic_con_quad)
plt.title('magnetic_flux_quad')

plt.show()

In [ ]:
def integrate_sims(fkt, a, b):
    return (b-a)/6*(fkt(a)+4*fkt((a+b)/2)+fkt(b))


In [ ]:
integrate_sims(cd.sun_angle, min_met, min_met+bin_step)

In [ ]:
point=ep.point_sources['1E1740-29']
def plot():   
    fig, ax = plt.subplots()

    ax.plot(cd.mean_time[:-1],point.separation_angle(cd.mean_time[:-1]))

    ax.set_xlabel('MET')
    ax.set_ylabel('Angle (deg)')
    return fig
plot()